# QA System | Language: Hindi | Content: History Books

This is a notebook demo of Question-Answering System on Hindi Language (History content). To Know detailed explanation, go to my blog: 

## Env Setup

In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4
!pip install sentence-transformers

## Haystack Pipeline Building (Manual)

In [4]:
import os
import json
import glob
import tqdm
from typing import List, Dict
from collections import defaultdict

from haystack.document_store.memory import InMemoryDocumentStore
from haystack.retriever.dense import DensePassageRetriever
from haystack.retriever.sparse import TfidfRetriever
from haystack.reader.farm import FARMReader
from haystack import Finder, finder


class QuestionAnsweringHindi:

    def __init__(self, textFilesDir: str, trainingFile: str):
        """

        :param textFilesDir: Path of dir which contain text (.txt) files (docs which supposed to be index)
        :param trainingData: Path of training json file (for fine-tuning purpose)
        """
        self.textFilesDir = textFilesDir
        self.trainingFile = trainingFile

    def get_data_haystack_format(self) -> List:
        """

        :return: Return a list of Haystack documents (specific format: refer this)
        """
        finalData = list()
        allFiles = glob.glob(os.path.join(self.textFilesDir, '*.txt'))
        for file in tqdm.tqdm(allFiles):
            tempDict = defaultdict()
            with open(file, 'r') as f:
                content = f.read()
            tempDict['text'] = content
            tempDict['meta'] = None
            finalData.append(dict(tempDict))
        return finalData

    def get_haystack_document_store(self, data, similarity_metric='cosine', do_index=True):
        """

        :param similarity_metric: Which metric to use to measure closeness either 'dot product' or 'cosine'
        :param do_index: Whether you want to index data after initializing document_store or not
        :param data: Haystack format data (list of dicts in predefined format)
        """
        # documentStore: Database to use to store & index the data i.e. ElasticSearch, InMemory, Faiss etc.
        documentStore = InMemoryDocumentStore(similarity_metric)
        if do_index:
            try:
                documentStore.write_documents(data)
                return documentStore
            except Exception as e:
                print(e)
        else:
            return documentStore

    def fine_tune_qa_model(self, outputModelName, epoch,
                           baseModel='sentence-transformers/msmarco-distilroberta-base-v2', use_gpu=True):
        """

        :param outputModelName: Fine-tuned model outputName provided by user
        :param epoch: no of epoch to fine-tune model
        :param baseModel: Base model on which you'd fine-tuning with custom data
        :param use_gpu: whether to use GPU or not (recommended to use)
        """
        reader = FARMReader(model_name_or_path=baseModel, use_gpu=-1)
        try:
            reader.train(data_dir=self.textFilesDir, train_filename=self.trainingFile, use_gpu=use_gpu, n_epochs=epoch,
                         save_dir=outputModelName)
            print('fine-tunning done!')
        except Exception as e:
            print(e)

    def get_haystack_retriever(self, document_store):
        """

        :param document_store: provide document_store where you indexed your data i.e. o/p of get_haystack_document_store()
        :return: return Haystack retriever object accordingly i.e BM25, tf-idf, DPR etc.
        """
        return TfidfRetriever(document_store=document_store)

    def get_haystack_reader(self, fine_tuned_model_path):
        """

        :param fine_tuned_model_path: location of model which you've just fine_tuned or any other you want to use i.e. sentence-transformers
        :return: return Haystack reader object.
        """
        return FARMReader(model_name_or_path=fine_tuned_model_path)

    def get_haystack_finder(self, reader, retriever):
        """

        :param reader: Haystack reader obj
        :param retriever: Haystack retriever
        :return: Haystack finder object
        """
        return Finder(reader, retriever)

    def get_answers(self, question, document_store, finder, n_retriever_result=10, n_reader_result=5):
        """

        :param question: User query
        :param finder: Haystack finder object
        :param n_retriever_result: No of results to extracted from retriever
        :param n_reader_result: No of results to be filtered with reader from retriever results
        :return: Python list of dict containing multiple properties of extracted answers.
        """
        results = finder.get_answers(question, top_k_retriever=n_retriever_result, top_k_reader=n_reader_result)
        return [{'answer': result['answer'], 'context': result['context'], 'startLoc': result['offset_start_in_doc'], 'endLoc': result['offset_end_in_doc'], 'docText':  self.get_haystack_doc_text_by_id(document_store, result['document_id']), 'probability': result['probability']} for
                result in results['answers']]

    def get_haystack_doc_text_by_id(self, document_store, docId):
        """

        :param document_store: Haystack Document Store Object
        :param docId: doc_id of Document
        :return: return the textual content of document.
        """
        return document_store.get_document_by_id(docId).to_dict()['text']


if __name__ == '__main__':
    pass

04/08/2021 11:20:16 - INFO - faiss.loader -   Loading faiss with AVX2 support.
04/08/2021 11:20:16 - INFO - faiss.loader -   Loading faiss.
04/08/2021 11:20:17 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
##pipeline running sequentially

#class initialise
qaObj = QuestionAnsweringHindi('path_to_text_dir', 'path_to_training_json_file')

#document store (I used InMemory())
ds = qaObj.get_haystack_document_store(data_to_index)

#fine tunning on custom data
qaObj.fine_tune_qa_model('path_to_output_model', 1)


#retriever initialise
retriever = qaObj.get_haystack_retriever(ds)

#reader initialise
reader = qaObj.get_haystack_reader('path_to_output_model')

#finder initialise
finder_new = qaObj.get_haystack_finder(reader, retriever)

#inferening
answers = qaObj.get_answers('कोलंबस ने द्वीप का नया नाम क्या रखा ?', ds, finder_new)

In [6]:
answers

[{'answer': 'सैन सैल्वाडोर',
  'context': 'गा।" कोलंबस ने गुआनाहानि में स्पेन का झंडा गाड़ दिया (उसने इस द्वीप का नया नाम सैन सैल्वाडोर, San Salvador रखा)। वहाँ उसने एक सार्वजनिक उपासना कराई और',
  'docText': ' 176 विश्व इतिहास के कुछ विषय अमरीकी मूलनिवासियों के साथ यूरोपीय लोगों की मुलाकात-यूरोप की काष्ठ ब्लॉक वाली एक छपाई, सोलहवीं सदी। Man में लिखा, "वे इतने ज्यादा उदार और सरल स्वभाव के लोग हैं कि अपना सब कुछ देने को तैयार हैं, वे कभी इनकार नहीं करते; बल्कि वे सदा बाँटने को तत्पर रहते हैं और इतना अधिक प्यार जताते हैं कि मानो उनका प्यार भरा कलेजा ही बाहर निकल आएगा।" कोलंबस ने गुआनाहानि में स्पेन का झंडा गाड़ दिया (उसने इस द्वीप का नया नाम सैन सैल्वाडोर, San Salvador रखा)। वहाँ उसने एक सार्वजनिक उपासना कराई और',
  'endLoc': 451,
  'probability': 0.9488971829414368,
  'startLoc': 438},
 {'answer': 'सॉलिडस (Solidus)',
  'context': 'वारा मुख्य रूप से मौद्रिक क्षेत्र में कुछ नए परिवर्तन किए गए। उसने सॉलिडस (Solidus) नाम का एक नया सिक्का चलाया जो 4.5 ग्राम शुद्ध सोने का बना हुआ था। ',
  'do